
# Homework 2, Problem 5 

Contributor: Nima Leclerc (nleclerc@seas.upenn.edu) 
Date: October 13, 2020 


In [ ]:
LOG_DIR = './logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

!if [ -f ngrok ] ; then echo "Ngrok already installed" ; else wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip > /dev/null 2>&1 && unzip ngrok-stable-linux-amd64.zip > /dev/null 2>&1 ; fi


In [ ]:
get_ipython().system_raw('./ngrok http 6006 &')

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print('Tensorboard Link: ' +str(json.load(sys.stdin)['tunnels'][0]['public_url']))"

Tensorboard Link: https://d498673af185.ngrok.io


In [ ]:
pip install tensorflow==1.15

     |████████████████████████████████| 412.3MB 41kB/s 
     |████████████████████████████████| 512kB 50.0MB/s 
     |████████████████████████████████| 3.8MB 53.1MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=de70c0ab808919e343f81ee8c72b898ed5ca514b7cab2c18861c790db09f1ec6
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully 

In [ ]:
# Code referenced from https://gist.github.com/gyglim/1f8dfb1b5c82627ae3efcfbbadb9f514
import tensorflow as tf
import numpy as np
import scipy.misc 
try:
    from StringIO import StringIO  # Python 2.7
except ImportError:
    from io import BytesIO         # Python 3.x


class Logger(object):
    
    def __init__(self, log_dir):
        """Create a summary writer logging to log_dir."""
        self.writer = tf.summary.FileWriter(log_dir)

    def scalar_summary(self, tag, value, step):
        """Log a scalar variable."""
        summary = tf.Summary(value=[tf.Summary.Value(tag=tag, simple_value=value)])
        self.writer.add_summary(summary, step)

    def image_summary(self, tag, images, step):
        """Log a list of images."""

        img_summaries = []
        for i, img in enumerate(images):
            # Write the image to a string
            try:
                s = StringIO()
            except:
                s = BytesIO()
            scipy.misc.toimage(img).save(s, format="png")

            # Create an Image object
            img_sum = tf.Summary.Image(encoded_image_string=s.getvalue(),
                                       height=img.shape[0],
                                       width=img.shape[1])
            # Create a Summary value
            img_summaries.append(tf.Summary.Value(tag='%s/%d' % (tag, i), image=img_sum))

        # Create and write Summary
        summary = tf.Summary(value=img_summaries)
        self.writer.add_summary(summary, step)
        
    def histo_summary(self, tag, values, step, bins=1000):
        """Log a histogram of the tensor of values."""

        # Create a histogram using numpy
        counts, bin_edges = np.histogram(values, bins=bins)

        # Fill the fields of the histogram proto
        hist = tf.HistogramProto()
        hist.min = float(np.min(values))
        hist.max = float(np.max(values))
        hist.num = int(np.prod(values.shape))
        hist.sum = float(np.sum(values))
        hist.sum_squares = float(np.sum(values**2))

        # Drop the start of the first bin
        bin_edges = bin_edges[1:]

        # Add bin edges and counts
        for edge in bin_edges:
            hist.bucket_limit.append(edge)
        for c in counts:
            hist.bucket.append(c)

        # Create and write Summary
        summary = tf.Summary(value=[tf.Summary.Value(tag=tag, histo=hist)])
        self.writer.add_summary(summary, step)
        self.writer.flush()

In [ ]:
# Comment the following code if not on colab
# bootstrap environment into place
from google.colab import auth
import datetime
auth.authenticate_user()

from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

import io
import os
from googleapiclient.http import MediaIoBaseDownload

# mount the user's google drive into the system
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# torch and torchvision imports
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim

logger = Logger('./logs')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assume that we are on a CUDA machine, then this should print a CUDA device:
print(device) 

cuda:0


In [42]:
# Reading in the dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16,
                                          shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=16,
                                         shuffle=False)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


# Defining the model
class View(nn.Module):
    def __init__(self,o):
        super().__init__()
        self.o = o

    def forward(self,x):
        return x.view(-1, self.o)
    
class allcnn_t(nn.Module):
    def __init__(self, c1=96, c2= 192):
        super().__init__()
        d = 0.5

        def convbn(ci,co,ksz,s=1,pz=0):
            return nn.Sequential(
                nn.Conv2d(ci,co,ksz,stride=s,padding=pz),
                nn.ReLU(True),
                nn.BatchNorm2d(co))

        self.m = nn.Sequential(
            nn.Dropout(0.2),
            convbn(3,c1,3,1,1),
            convbn(c1,c1,3,1,1),
            convbn(c1,c1,3,2,1),
            nn.Dropout(d),
            convbn(c1,c2,3,1,1),
            convbn(c2,c2,3,1,1),
            convbn(c2,c2,3,2,1),
            nn.Dropout(d),
            convbn(c2,c2,3,1,1),
            convbn(c2,c2,3,1,1),
            convbn(c2,10,1,1),
            nn.AvgPool2d(8),
            View(10))

        print('Num parameters: ', sum([p.numel() for p in self.m.parameters()]))

    def forward(self, x):
        return self.m(x)

# The training loop

def train(net, optimizer, criterion, train_loader, test_loader, epochs, model_name, plot):
    model = net.to(device)
    total_step = len(train_loader)
    overall_step = 0
    for epoch in range(epochs):
        correct = 0
        total = 0
        for i, (images, labels) in enumerate(train_loader):
            # Move tensors to configured device
            images = images.to(device)
            labels = labels.to(device)
            #Forward Pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if (i+1) % 1000 == 0:
              print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, i+1, total_step, loss.item()))
            if plot:
              info = { ('loss_' + model_name): loss.item() }

              for tag, value in info.items():
                logger.scalar_summary(tag, value, overall_step+1)



    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for i, (images, labels) in enumerate(test_loader):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the test images: {} %'.format(100 * correct / total))

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
model = allcnn_t().to(device)
epoch_1 = 40 
epoch_2 = 40 
epoch_3 = 20 
alpha_1 = 0.1
alpha_2 = 0.01 
alpha_3 = 0.001 
decay = 0.001 
decay_final = 0.0001 
criterion = nn.CrossEntropyLoss()

Num parameters:  1667166


(a) The model is trained on the CIFAR-10 dataset using a weight-decay of 0.001 and momentum of 0.9. The learning rate for the first 40 epochs is set to 0.1, for the second 40 epochs is 0.01, and 0.001 for the final 20 epochs. These will be staggered into 3 seperate rounds of training.  

First round of training: alpha = 0.1, beta = 0.9, weight decay = 0.001 for 40 epochs.  

In [ ]:
optimizer1 = optim.SGD(model.parameters(), lr=alpha_1, momentum=0.9, weight_decay=0.001)
train(model, optimizer1, criterion, trainloader, testloader, epoch_1, 'cnn', True)

Epoch [1/40], Step [1000/3125], Loss: 1.5895
Epoch [1/40], Step [2000/3125], Loss: 1.5407
Epoch [1/40], Step [3000/3125], Loss: 1.3922
Epoch [2/40], Step [1000/3125], Loss: 1.3556
Epoch [2/40], Step [2000/3125], Loss: 1.6914
Epoch [2/40], Step [3000/3125], Loss: 1.4758
Epoch [3/40], Step [1000/3125], Loss: 2.0685
Epoch [3/40], Step [2000/3125], Loss: 1.8247
Epoch [3/40], Step [3000/3125], Loss: 1.5538
Epoch [4/40], Step [1000/3125], Loss: 1.2449
Epoch [4/40], Step [2000/3125], Loss: 1.6770
Epoch [4/40], Step [3000/3125], Loss: 1.4262
Epoch [5/40], Step [1000/3125], Loss: 1.7119
Epoch [5/40], Step [2000/3125], Loss: 1.2999
Epoch [5/40], Step [3000/3125], Loss: 1.4009
Epoch [6/40], Step [1000/3125], Loss: 1.7566
Epoch [6/40], Step [2000/3125], Loss: 1.5592
Epoch [6/40], Step [3000/3125], Loss: 1.0612
Epoch [7/40], Step [1000/3125], Loss: 1.0676
Epoch [7/40], Step [2000/3125], Loss: 1.6733
Epoch [7/40], Step [3000/3125], Loss: 1.3834
Epoch [8/40], Step [1000/3125], Loss: 1.4796
Epoch [8/4

A test accuracy of 45.98 % in the first 40 epochs. 

In [ ]:
torch.save(model.state_dict(), "/content/allcnn_v0")

Second round of training: alpha = 0.01, beta = 0.9, weight decay = 0.001 for 40 epochs.

In [ ]:
optimizer2 = optim.SGD(model.parameters(), lr=alpha_2, momentum=0.9, weight_decay=0.001)
train(model, optimizer2, criterion, trainloader, testloader, epoch_2, 'cnn', True)

Epoch [1/40], Step [1000/3125], Loss: 0.8973
Epoch [1/40], Step [2000/3125], Loss: 0.7319
Epoch [1/40], Step [3000/3125], Loss: 1.3257
Epoch [2/40], Step [1000/3125], Loss: 0.9179
Epoch [2/40], Step [2000/3125], Loss: 0.9999
Epoch [2/40], Step [3000/3125], Loss: 0.7893
Epoch [3/40], Step [1000/3125], Loss: 0.6011
Epoch [3/40], Step [2000/3125], Loss: 0.9540
Epoch [3/40], Step [3000/3125], Loss: 0.6341
Epoch [4/40], Step [1000/3125], Loss: 0.3015
Epoch [4/40], Step [2000/3125], Loss: 0.6082
Epoch [4/40], Step [3000/3125], Loss: 0.8472
Epoch [5/40], Step [1000/3125], Loss: 0.7760
Epoch [5/40], Step [2000/3125], Loss: 0.3909
Epoch [5/40], Step [3000/3125], Loss: 0.8702
Epoch [6/40], Step [1000/3125], Loss: 0.3182
Epoch [6/40], Step [2000/3125], Loss: 0.4683
Epoch [6/40], Step [3000/3125], Loss: 1.0927
Epoch [7/40], Step [1000/3125], Loss: 0.6552
Epoch [7/40], Step [2000/3125], Loss: 0.7552
Epoch [7/40], Step [3000/3125], Loss: 0.5908
Epoch [8/40], Step [1000/3125], Loss: 0.7532
Epoch [8/4

Second round of training yields a test accuracy of 81.24 % after a total of 80 epochs.

In [ ]:
torch.save(model.state_dict(), "/content/allcnn_v0")

Third round of training: alpha = 0.001, beta = 0.9, weight decay = 0.001 for 20 epochs.

In [ ]:
optimizer3 = optim.SGD(model.parameters(), lr=alpha_3, momentum=0.9, weight_decay=0.001)
train(model, optimizer3, criterion, trainloader, testloader, epoch_3, 'cnn', True)

Epoch [1/20], Step [1000/3125], Loss: 0.0706
Epoch [1/20], Step [2000/3125], Loss: 0.3540
Epoch [1/20], Step [3000/3125], Loss: 0.0744
Epoch [2/20], Step [1000/3125], Loss: 0.0288
Epoch [2/20], Step [2000/3125], Loss: 0.0456
Epoch [2/20], Step [3000/3125], Loss: 0.2865
Epoch [3/20], Step [1000/3125], Loss: 0.0745
Epoch [3/20], Step [2000/3125], Loss: 0.1764
Epoch [3/20], Step [3000/3125], Loss: 0.0383
Epoch [4/20], Step [1000/3125], Loss: 0.0484
Epoch [4/20], Step [2000/3125], Loss: 0.1845
Epoch [4/20], Step [3000/3125], Loss: 0.0880
Epoch [5/20], Step [1000/3125], Loss: 0.1191
Epoch [5/20], Step [2000/3125], Loss: 0.0625
Epoch [5/20], Step [3000/3125], Loss: 0.0511
Epoch [6/20], Step [1000/3125], Loss: 0.0272
Epoch [6/20], Step [2000/3125], Loss: 0.0120
Epoch [6/20], Step [3000/3125], Loss: 0.0143
Epoch [7/20], Step [1000/3125], Loss: 0.0068
Epoch [7/20], Step [2000/3125], Loss: 0.0653
Epoch [7/20], Step [3000/3125], Loss: 0.0995
Epoch [8/20], Step [1000/3125], Loss: 0.0100
Epoch [8/2

Third round of training yields a test accuracy of 83.18 % after a total of 100 epochs.

In [ ]:
torch.save(model.state_dict(), "/content/allcnn_v0")

Fourth round of training: alpha = 0.0005, beta = 0.9, weight decay = 0.001 for 20 epochs.

In [ ]:
optimizer4 = optim.SGD(model.parameters(), lr=0.0005, momentum=0.9, weight_decay=0.001)
train(model, optimizer4, criterion, trainloader, testloader, epoch_3, 'cnn', True)

Epoch [1/20], Step [1000/3125], Loss: 0.0042
Epoch [1/20], Step [2000/3125], Loss: 0.0044
Epoch [1/20], Step [3000/3125], Loss: 0.0017
Epoch [2/20], Step [1000/3125], Loss: 0.0039
Epoch [2/20], Step [2000/3125], Loss: 0.0084
Epoch [2/20], Step [3000/3125], Loss: 0.0097
Epoch [3/20], Step [1000/3125], Loss: 0.0341
Epoch [3/20], Step [2000/3125], Loss: 0.0041
Epoch [3/20], Step [3000/3125], Loss: 0.0153
Epoch [4/20], Step [1000/3125], Loss: 0.0107
Epoch [4/20], Step [2000/3125], Loss: 0.0004
Epoch [4/20], Step [3000/3125], Loss: 0.0029
Epoch [5/20], Step [1000/3125], Loss: 0.0033
Epoch [5/20], Step [2000/3125], Loss: 0.0022
Epoch [5/20], Step [3000/3125], Loss: 0.0087
Epoch [6/20], Step [1000/3125], Loss: 0.0077
Epoch [6/20], Step [2000/3125], Loss: 0.0050
Epoch [6/20], Step [3000/3125], Loss: 0.0072
Epoch [7/20], Step [1000/3125], Loss: 0.0099
Epoch [7/20], Step [2000/3125], Loss: 0.0064
Epoch [7/20], Step [3000/3125], Loss: 0.0020
Epoch [8/20], Step [1000/3125], Loss: 0.0030
Epoch [8/2

Second round of training yields a test accuracy of 84.09% after a total of 120 epochs

---



In [ ]:
torch.save(model.state_dict(), "/content/allcnn_v0")

Plotted below are the training losses, validation losses, and errors as a function of number of epochs. 

(b) We can now get the gradient dx for a few input images. 

In [104]:
from torch.autograd import Variable
for i, (images, labels) in enumerate(testloader):
    # Move tensors to configured device
    images = images.to(device)
    labels = labels.to(device)
    x = Variable(images, requires_grad=True)
criterion = nn.CrossEntropyLoss()   
yh = model(x)
loss = criterion(yh,labels)
y_test =  Variable(loss, requires_grad=True)
y_test.backward(images)
#yh.backward(images)
#dx = x[0:].grad.data.clone()

RuntimeError: ignored

In [82]:
import torch
from torch.autograd import Variable

x = Variable(torch.ones(10), requires_grad=True)
y = x * Variable(torch.linspace(1, 10, 10), requires_grad=False)
y.backward(torch.ones(10))
print(x.grad)

tensor([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])
